# Running LazyPredict subsetting for Top features identified via XGBoost & SHAP analysis


In [2]:
#
#import get_lazy_regresor()
import importlib.util
import sys

def import_function_from_path(file_path, function_name):
    # Load the module from the file path
    spec = importlib.util.spec_from_file_location("module_name", file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules["module_name"] = module
    spec.loader.exec_module(module)
    
    # Retrieve the function from the loaded module
    func = getattr(module, function_name)
    return func

PATH_TO_SCRIPT = '/Users/masaver/Desktop/masaver/data_science_projects/sep24_bds_int_medical/notebooks/helpers/LazyPredict.py'
function_name = 'get_lazy_regressor'

get_lazy_regressor = import_function_from_path( PATH_TO_SCRIPT , function_name )

PATH_TO_SRC = '/Users/masaver/Desktop/masaver/data_science_projects/sep24_bds_int_medical'
sys.path.append( PATH_TO_SRC )

# Load other requiered libraries
import os
import pickle
import pandas as pd
import numpy as np
from pprint import pprint as pp
from sklearn.model_selection import train_test_split

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

# Import the preprocessign pipeline
from pipelines import *

In [3]:
# Read and display the train and test data
data_file = os.path.join('..', '..', 'data', 'raw', 'train.csv')
df_train = pd.read_csv(data_file, index_col = 0 , parse_dates = True )
display( df_train.head() )


,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,p01,06:10:00,NaN,NaN,9.60,NaN,NaN,9.70,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.40
p01_1,p01,06:25:00,NaN,NaN,9.70,NaN,NaN,9.20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.80
p01_2,p01,06:40:00,NaN,NaN,9.20,NaN,NaN,8.70,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.50
p01_3,p01,06:55:00,NaN,NaN,8.70,NaN,NaN,8.40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.80
p01_4,p01,07:10:00,NaN,NaN,8.40,NaN,NaN,8.10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.70


In [4]:
# Load the pickle file with the previous XGboost & SHAP analysis
with open('./models/03_xgboost_HyperOpt.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

f.close()
pp(loaded_dict['top_features'])

['bg-0:00',
 'bg-0:05',
 'hr-0:00',
 'bg-0:15',
 'bg-0:20',
 'bg-0:10',
 'bg-0:25',
 'cals-1:50',
 'cals-5:20',
 'bg-0:30',
 'cals-5:25',
 'cals-5:05',
 'hr-0:05',
 'insulin-0:20',
 'bg-0:35']


In [5]:
# Split the data into Features and Target variables, 
# and Standarize the features with the preprocessing pipelines
X = df_train.drop( 'bg+1:00' , axis = 1 )
X = X[ loaded_dict['top_features'] ]
y = df_train['bg+1:00']

# Train Test Split
x_train,x_test,y_train,y_test = train_test_split( X , y , test_size=0.2 , random_state=17 )
data_pipe = pipeline_s
x_train_s = data_pipe.fit_transform( x_train )
x_test_s = data_pipe.transform( x_test )

display( x_train_s )
display( x_test_s )


,bg-0:00,bg-0:05,hr-0:00,bg-0:15,bg-0:20,bg-0:10,bg-0:25,cals-1:50,cals-5:20,bg-0:30,cals-5:25,cals-5:05,hr-0:05,insulin-0:20,bg-0:35
id,,,,,,,,,,,,,,,
p11_1931,0.48,0.58,-0.15,0.74,0.91,0.68,0.91,-0.42,-0.36,0.84,-0.40,-0.41,-0.36,-0.19,0.74
p10_9422,-0.80,-0.73,-0.29,-0.66,-0.90,-0.66,-1.00,-0.44,-0.42,-1.00,-0.42,-0.42,-0.91,-0.18,-1.00
p06_8273,-1.30,-1.26,1.34,-1.20,-1.14,-1.23,-1.09,-0.14,2.92,-1.03,2.02,1.32,1.23,-0.10,-1.03
p12_17133,0.18,0.24,1.74,0.34,0.34,0.31,0.34,-0.05,-0.26,0.34,-0.22,-0.31,1.86,-0.02,0.27
p03_2346,0.01,0.17,-0.54,0.04,0.17,0.17,0.04,0.10,-0.45,-0.03,-0.45,-0.45,-0.38,0.00,-0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p02_17172,0.84,0.88,-0.42,0.91,0.88,0.88,0.84,-1.09,-0.21,0.87,-0.82,-0.78,-0.29,-0.09,0.94
p10_23964,0.98,1.14,0.45,1.18,1.01,1.18,0.71,1.90,0.86,0.31,0.13,-0.20,0.89,0.05,-0.16
p03_7966,-0.63,-0.63,-0.90,-0.53,-0.49,-0.56,-0.50,-0.47,-0.45,-0.46,-0.45,-0.45,-0.56,-0.11,-0.43


,bg-0:00,bg-0:05,hr-0:00,bg-0:15,bg-0:20,bg-0:10,bg-0:25,cals-1:50,cals-5:20,bg-0:30,cals-5:25,cals-5:05,hr-0:05,insulin-0:20,bg-0:35
id,,,,,,,,,,,,,,,
p11_17351,-0.09,-0.09,-0.83,0.01,0.21,-0.09,0.54,-0.42,0.92,0.64,-0.12,-0.41,-1.00,-0.12,0.64
p10_15385,-0.80,-0.90,-0.83,-0.86,-0.80,-0.90,-0.73,-0.44,-0.41,-0.73,-0.42,-0.42,-1.00,-0.16,-0.69
p06_2496,0.04,0.01,-0.63,-0.06,-0.08,-0.03,-0.10,-0.38,-0.38,-0.13,-0.38,-0.39,-0.51,-0.11,-0.13
p10_8410,0.14,0.07,0.64,-0.03,-0.23,0.04,-0.46,0.80,2.12,-0.66,3.15,0.02,0.47,-0.11,-0.83
p02_2562,-0.36,-0.43,0.64,-0.49,-0.49,-0.49,-0.53,0.80,2.12,-0.59,3.15,0.02,1.14,-0.13,-0.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p02_24761,-0.70,-0.70,0.51,-0.63,-0.63,-0.60,-0.63,0.23,-0.55,-0.59,-0.55,-0.54,0.86,-0.14,-0.59
p10_22597,0.58,0.61,0.12,0.54,0.51,0.58,0.37,0.05,-0.42,0.21,-0.40,-0.39,-0.02,-0.13,0.17
p03_2231,-0.36,-0.36,-1.41,-0.33,-0.33,-0.36,-0.33,-0.45,-0.35,-0.26,-0.25,-0.30,-1.41,-0.13,-0.13


In [6]:
# Display descriptive statistics
display( x_train_s.describe() )
display( x_test_s.describe() )

,bg-0:00,bg-0:05,hr-0:00,bg-0:15,bg-0:20,bg-0:10,bg-0:25,cals-1:50,cals-5:20,bg-0:30,cals-5:25,cals-5:05,hr-0:05,insulin-0:20,bg-0:35
count,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00,141619.00
mean,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.03,-2.03,-2.23,-2.03,-2.03,-2.03,-2.03,-1.17,-1.15,-2.03,-1.14,-1.15,-2.32,-0.19,-2.03
25%,-0.73,-0.73,-0.81,-0.73,-0.73,-0.73,-0.73,-0.47,-0.45,-0.73,-0.45,-0.45,-0.81,-0.16,-0.73
50%,-0.19,-0.19,-0.10,-0.19,-0.19,-0.19,-0.19,-0.40,-0.38,-0.19,-0.38,-0.39,-0.10,-0.12,-0.19
75%,0.54,0.54,0.59,0.54,0.54,0.54,0.54,-0.04,-0.04,0.54,-0.04,-0.04,0.59,-0.08,0.54
max,6.54,6.53,5.86,6.52,6.53,6.53,6.53,12.79,13.37,6.52,13.28,13.36,5.87,47.42,6.52


,bg-0:00,bg-0:05,hr-0:00,bg-0:15,bg-0:20,bg-0:10,bg-0:25,cals-1:50,cals-5:20,bg-0:30,cals-5:25,cals-5:05,hr-0:05,insulin-0:20,bg-0:35
count,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00,35405.00
mean,-0.00,0.00,-0.01,0.00,0.00,-0.00,0.00,0.01,0.00,0.00,0.00,-0.00,-0.00,0.01,0.00
std,1.00,1.01,0.99,1.00,1.01,1.01,1.01,1.02,1.00,1.01,1.00,1.00,1.00,1.09,1.01
min,-2.03,-2.03,-1.98,-2.03,-2.03,-2.03,-2.03,-1.17,-1.15,-2.03,-1.14,-1.15,-2.10,-0.19,-2.03
25%,-0.73,-0.73,-0.82,-0.73,-0.73,-0.73,-0.73,-0.47,-0.45,-0.73,-0.45,-0.45,-0.81,-0.16,-0.73
50%,-0.19,-0.19,-0.11,-0.19,-0.19,-0.19,-0.19,-0.40,-0.38,-0.19,-0.38,-0.39,-0.10,-0.12,-0.19
75%,0.54,0.54,0.58,0.54,0.54,0.54,0.54,-0.03,-0.04,0.54,-0.05,-0.05,0.60,-0.08,0.54
max,6.37,6.32,5.80,6.22,6.31,6.27,6.40,13.36,13.37,6.49,13.28,13.36,5.78,25.74,6.48


In [7]:
# Run a Lazy Regressor
reg = get_lazy_regressor()
models, predictions = reg.fit( x_train_s , x_test_s , y_train , y_test )

 97%|█████████▋| 38/39 [13:56<00:49, 49.15s/it] 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000956 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3735
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 15
[LightGBM] [Info] Start training from score 8.273489


100%|██████████| 39/39 [13:57<00:00, 21.47s/it]


In [8]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
XGBRegressor,0.57,0.57,1.97,0.41
LGBMRegressor,0.56,0.56,1.98,0.67
HistGradientBoostingRegressor,0.56,0.56,1.99,0.87
MLPRegressor,0.56,0.56,2.00,26.77
ExtraTreesRegressor,0.55,0.55,2.00,34.76
GradientBoostingRegressor,0.54,0.54,2.04,20.05
SVR,0.52,0.52,2.07,439.33
BaggingRegressor,0.52,0.52,2.07,10.17
LassoLarsIC,0.49,0.49,2.14,0.08
